# RAG ETL Pipeline для Yandex Handbook

**Архитектура:**
- **Child chunks** (450 токенов) → Qdrant для точного поиска (HYBRID: dense + sparse)
- **Parent chunks** (1500 токенов) → Redis для контекста
- **ParentDocumentRetriever** связывает chunks

**Установка пакетов:**
```bash
uv add langchain-core langchain-community langchain-openai langchain-text-splitters langchain-qdrant
uv add qdrant-client python-dotenv tiktoken tqdm
```

## 1. Переменные окружения и настройки

In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv

# Добавляем родительскую директорию в path для импорта settings и llm_service
sys.path.append('..')

# Явная загрузка .env из родительской папки
env_path = Path('..') / '.env'
print(f"🔍 Загрузка .env из: {env_path.absolute()}")
load_dotenv(dotenv_path=env_path, override=True)

from settings import get_settings
from llm_service.llm_client import LLMClient

cfg = get_settings()
client = LLMClient(provider="openai")

print("🚀 Настройки загружены через LLMSettings")
print(f"   • Провайдер: {cfg.default_provider}")
print(f"   • Chat Model: {cfg.openai_chat_model_name}")
print(f"   • Embedding Model: {cfg.openai_embedding_model_name}")
print(f"   • API Base: {cfg.openai_api_base}")
print(f"   • Qdrant URL: {os.getenv('QDRANT_URL')}")
print(f"   • Redis URL: {os.getenv('REDIS_URL')}")

# DEBUG: Проверка ключа (выводим только часть для безопасности)
key = os.getenv('OPENAI_API_KEY')
if key:
    print(f"✅ Ключ в env найден: {key[:5]}...{key[-4:]}")
else:
    print("❌ Ключ OPENAI_API_KEY НЕ НАЙДЕН в os.environ")

🔍 Загрузка .env из: /home/anton-admin/Загрузки/develop/education/agent_proj/lifelong_learning_assistant/rag/notebook/../.env


2026-01-15 11:15:47 | INFO | llm_service.llm_client | Инициализация LLM-клиента: провайдер=openai


🚀 Настройки загружены через LLMSettings
   • Провайдер: openai
   • Chat Model: gpt-4o-mini
   • Embedding Model: text-embedding-3-large
   • API Base: https://api.ai-mediator.ru/v1
   • Qdrant URL: http://localhost:6333
   • Redis URL: redis://localhost:6379
✅ Ключ в env найден: sk-yZ...Ckxw


## 2. Импорты

In [2]:
import tiktoken
import re
import hashlib
import json
from typing import Dict, Any
from pathlib import Path
from uuid import uuid4

# LangChain
from langchain_core.documents import Document
from langchain_core.load import dumpd, loads
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_qdrant.fastembed_sparse import FastEmbedSparse
from langchain_community.storage import RedisStore

# Qdrant
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams, SparseVectorParams, SparseIndexParams

# Progress bar
from tqdm.auto import tqdm

print("📦 Импорты загружены")

📦 Импорты загружены


/home/anton-admin/Загрузки/develop/education/agent_proj/lifelong_learning_assistant/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3. Функции очистки текста

In [3]:
def clean_document_text(text: str) -> str:
    """Очистка Markdown-текста от ссылок, файлов, форматирования"""
    if not text:
        return ""
    
    # Удаление ссылок [текст](url) → текст
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    
    # Удаление изображений ![alt](src)
    text = re.sub(r'!\[[^\]]*\].*?\)', '', text)
    
    # Удаление URL
    text = re.sub(r'https?://\S+', '', text)
    
    # Удаление форматирования **текст** → текст
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    
    # Удаление HTML тегов
    text = re.sub(r'<[^>]+>', '', text)
    
    # Удаление лишних пробелов
    text = re.sub(r'\s{2,}', ' ', text)
    text = text.strip()
    
    # Lowercase для лучшего поиска
    return text.lower()

## 4. Функции для метаданных

In [4]:
def stable_parent_id(meta: Dict[str, Any], text: str) -> str:
    """Детерминированный parent_id"""
    base = (
        str(meta.get("source", "")) + "|" +
        str(meta.get("H1", "")) + "|" + 
        str(meta.get("H2", "")) + "|" + 
        str(meta.get("H3", "")) + "|" +
        (text[:128])
    ).encode("utf-8")
    return hashlib.md5(base).hexdigest()

## 5. Загрузка документов (БЕЗ очистки)

In [5]:
# Настройка токенизатора
encoder = tiktoken.get_encoding("cl100k_base")
tiktoken_len = lambda text: len(encoder.encode(text))

# Путь к данным
DATA_PATH = "../../data/yandex-handbook-downloaded"

# Загружаем документы БЕЗ очистки (нужны заголовки ##, ###)
print(f"📂 Загружаем документы из: {DATA_PATH}")
md_files = list(Path(DATA_PATH).glob("*.md"))
print(f"   • Найдено {len(md_files)} .md файлов")

documents = []
for file_path in md_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    doc = Document(
        page_content=content,
        metadata={
            'source': str(file_path),
            'filename': file_path.name,
            'total_tokens': tiktoken_len(content)
        }
    )
    documents.append(doc)

print(f"✅ Загружено {len(documents)} документов")

📂 Загружаем документы из: ../../data/yandex-handbook-downloaded
   • Найдено 68 .md файлов
✅ Загружено 68 документов


## 6. Загрузка index.json для названий документов

In [6]:
# Загружаем index.json
INDEX_PATH = Path(DATA_PATH) / "index.json"

with open(INDEX_PATH, 'r', encoding='utf-8') as f:
    index_data = json.load(f)

# Создаем маппинг filename -> title
filename_to_title = {}
for doc_id, doc_info in index_data.items():
    filename = doc_id + '.md'
    title = doc_info.get('title', '')
    filename_to_title[filename] = title

print(f"📖 Загружено {len(filename_to_title)} названий документов")

📖 Загружено 68 названий документов


## 7. Настройка сплиттеров

In [7]:
# Markdown Header Splitter для parent chunks
headers_to_split_on = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False
)

# Recursive splitter для больших parent chunks
parent_recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
    keep_separator=True
)

# Child splitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=70,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""],
    keep_separator=True
)

## 8. Создание parent chunks

In [8]:
parent_docs = []
for doc in tqdm(documents, desc="Создание parent chunks"):
    doc_filename = doc.metadata.get('filename', '')
    doc_title = filename_to_title.get(doc_filename, '')
    
    md_chunks = markdown_splitter.split_text(doc.page_content or "")
    
    for chunk in md_chunks:
        merged_meta = {**(doc.metadata or {}), **(chunk.metadata or {})}
        merged_meta["doc_title"] = doc_title
        
        headers = [merged_meta.get(k, "") for k in ("H1", "H2", "H3") if k in merged_meta]
        merged_meta["breadcrumbs"] = " / ".join(headers) if headers else ""
        
        path_parts = [doc_title] if doc_title else []
        path_parts.extend(headers)
        merged_meta["path"] = " / ".join(path_parts) if path_parts else ""
        
        merged_meta["level"] = 3 if "H3" in merged_meta else 2 if "H2" in merged_meta else 1 if "H1" in merged_meta else 0
        
        # Очищаем текст chunk
        chunk.page_content = clean_document_text(chunk.page_content)
        
        # Создаем parent_id
        merged_meta["parent_id"] = stable_parent_id(merged_meta, chunk.page_content)
        chunk.metadata = merged_meta
        
        if tiktoken_len(chunk.page_content) > 1500:
            sub_chunks = parent_recursive_splitter.split_documents([chunk])
            for sub in sub_chunks:
                sub.metadata = merged_meta.copy()
                sub.metadata["parent_id"] = stable_parent_id(sub.metadata, sub.page_content)
            parent_docs.extend(sub_chunks)
        else:
            parent_docs.append(chunk)

print(f"✅ Создано {len(parent_docs)} parent chunks")

Создание parent chunks: 100%|██████████| 68/68 [00:00<00:00, 87.12it/s] 

✅ Создано 923 parent chunks


## 9. Настройка Qdrant и Redis

In [9]:
# Используем LLMClient для создания эмбеддингов
embeddings = client.create_embeddings()

# Sparse embeddings (BM25)
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

print("✨ Эмбеддинги настроены (через LLMClient)")

# Qdrant
qdrant_url = os.getenv('QDRANT_URL', 'http://localhost:6333')
collection_name = os.getenv('QDRANT_COLLECTION_NAME', 'yandex_handbook_child_chunks')
qdrant_client = QdrantClient(url=qdrant_url)

try:
    qdrant_client.delete_collection(collection_name)
    print(f"🗑️ Удалена старая коллекция {collection_name}")
except:
    pass

# Получаем размерность вектора из модели автоматически
sample_vector = embeddings.embed_query("test")
vector_size = len(sample_vector)
print(f"📏 Автоматическое определение размерности: {vector_size}")

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=vector_size, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams(
            index=SparseIndexParams(on_disk=False),
            modifier=models.Modifier.IDF,
        )
    },
)

vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name=collection_name,
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID,
    vector_name="dense",
    sparse_vector_name="sparse",
)

print(f"✅ Qdrant настроен: {collection_name}")

# Redis
redis_url = os.getenv('REDIS_URL', 'redis://localhost:6379')
parent_store = RedisStore(
    redis_url=redis_url,
    namespace="rag:parents"
)

print(f"✅ Redis настроен: {redis_url}")

2026-01-15 11:16:18 | INFO | llm_service.llm_client | start:create_embeddings провайдер=openai


✨ Эмбеддинги настроены (через LLMClient)
🗑️ Удалена старая коллекция yandex_handbook_child_chunks
📏 Автоматическое определение размерности: 3072
✅ Qdrant настроен: yandex_handbook_child_chunks
✅ Redis настроен: redis://localhost:6379


## 10. Загрузка данных

In [10]:
print("📥 Загружаем данные...")

def doc_to_bytes(doc: Document) -> bytes:
    as_dict = dumpd(doc)
    return json.dumps(as_dict, ensure_ascii=False).encode("utf-8")

# 1. Redis
parent_ids = []
batch = []
for doc in tqdm(parent_docs, desc="Загрузка в Redis"):
    pid = doc.metadata["parent_id"]
    batch.append((pid, doc_to_bytes(doc)))
    parent_ids.append(pid)
    if len(batch) >= 1000:
        parent_store.mset(batch)
        batch.clear()
if batch: parent_store.mset(batch)

print(f"✅ Загружено {len(parent_ids)} parent chunks в Redis")

# 2. Child chunks -> Qdrant
child_docs = []
for parent in tqdm(parent_docs, desc="Создание child chunks"):
    parent_id = parent.metadata["parent_id"]
    children = child_splitter.split_documents([parent])
    for child in children:
        child.metadata.update(parent.metadata)
        child.metadata["parent_id"] = parent_id
        child.metadata["child_id"] = str(uuid4())
        child.metadata["is_child_chunk"] = True
        path = child.metadata.get("path", "")
        if path: child.page_content = f"{path}\n\n{child.page_content}"
        child_docs.append(child)

print(f"✅ Создано {len(child_docs)} child chunks")

if child_docs:
    batch_size = 100
    for i in tqdm(range(0, len(child_docs), batch_size), desc="Загрузка в Qdrant"):
        batch = child_docs[i:i+batch_size]
        vectorstore.add_documents(batch)

collection_info = qdrant_client.get_collection(collection_name)
print(f"\n📊 Итого: Qdrant={collection_info.points_count}, Redis={len(parent_ids)}")

📥 Загружаем данные...


Загрузка в Redis: 100%|██████████| 923/923 [00:00<00:00, 9918.69it/s]


✅ Загружено 923 parent chunks в Redis


Создание child chunks: 100%|██████████| 923/923 [00:01<00:00, 576.75it/s]


✅ Создано 2071 child chunks


Загрузка в Qdrant: 100%|██████████| 21/21 [01:27<00:00,  4.16s/it]


📊 Итого: Qdrant=2071, Redis=923
